In [1]:
import sys, os
from Deconvolution.BLADE import Framework
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from sklearn.svm import SVR
from sklearn.svm import NuSVR

from sklearn.metrics import mean_squared_error as mse
import pandas as pd

# modules for visualization
import qgrid
from matplotlib import pyplot as plt
import seaborn as sns

### Run BLADE with TCGA bulk and Puram scRNA-seq reference


#### Application of deconvolution methods

From here, we will apply the following three methods for further performance comparison:
1. BLADE (estimation of cellular fraction + group-mode/high-resolution-mode purification)
2. NNLS (estimation of fraction)
3. SVR followed by NNLS (estimation of fraction + group-mode purification) - similar to CIBERSORTx


##### 1. Application of BLADE

These are the key parameters used in BLADE (note that there is default settings of these parameters, if not specified):
- Hyperparameters (`hyperpars`): `Alpha`, `Alpha0`, `Kappa0` and `SigmaY`, each of which can be defined as a list of options. BLADE takes an empirical Bayes approach to find the optimal parameter set given the all possible combinations. 
- `Nrep`: Number of repeat for evaluating each parameter configuration.
- `Nrepfinal`: Number of repeated optimizations for the final parameter set.
- `Njob`: Number of parallel jobs.

## Prepare signature matrix as required

In [2]:
df_Puram_std = pd.read_csv("/home/cke/Puram/HNSCC2PuramGSE103322_qc_std.tsv",sep='\t',index_col=0)
df_Puram_mean = pd.read_csv("/home/cke/Puram/HNSCC2PuramGSE103322_qc_mean.tsv",sep='\t',index_col=0)
marker_genes = pd.read_csv("/home/cke/Puram/top100DEGs_pseudobulk.txt",header=None).iloc[0,:]

marker_genes = marker_genes.drop_duplicates()
df_Puram_std_filtered = df_Puram_std.loc[marker_genes,:]
df_Puram_mean_filtered = df_Puram_mean.loc[marker_genes,:]

df_pseudo = pd.read_csv("/home/cke/Puram/Puram_pseudobulk_fromraw.tsv",sep='\t',index_col=0).T
df_Puram_mean_log2 = np.log2(df_Puram_mean_filtered+1)

merge_genes_mean = pd.merge(df_Puram_mean_log2,df_pseudo,left_index=True,right_index=True,how='inner')
merge_genes_std = pd.merge(df_Puram_std_filtered,df_pseudo,left_index=True,right_index=True,how='inner')

print("Get mean and std exp!")

#simple tumor cell type setup, there are 10 annotated cell types
df_TCGA_shared = merge_genes_mean.iloc[:,10:]
df_shared_mean = merge_genes_mean.iloc[:,:10]
df_shared_std = merge_genes_std.iloc[:,:10]

print("Get common genes! ",df_shared_mean.shape[0])
print("cell types: ",df_shared_mean.shape[1])
print("bulk samples: ",df_TCGA_shared.shape[1])


Get mean and std exp!
Get common genes!  880
cell types:  10
bulk samples:  17


In [62]:
df_Puram_mean

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,T cell,myocyte,other,tumor
C9orf152,0.000000,0.017683,0.028524,0.006479,0.000000,0.000000,0.000989,0.000000,0.126178,0.021723
RPS11,5.841971,5.327779,5.893488,5.699910,5.739950,4.537083,4.093909,5.563400,5.725872,6.394412
ELMO2,0.208097,0.674968,0.542106,0.621450,0.564073,0.791185,0.553634,0.449079,0.500478,0.743162
CREB3L1,0.000000,0.080368,0.015094,0.134395,0.000000,0.000000,0.000375,0.000000,0.352287,0.004988
PNMA1,0.230034,1.543609,0.737776,1.045224,0.220860,0.456808,0.351467,0.395577,0.281493,0.815725
...,...,...,...,...,...,...,...,...,...,...
PIK3IP1,0.223408,0.426315,0.865399,0.635732,1.446469,1.883503,3.408460,0.126706,1.086586,1.125250
SNRPD2,2.039287,1.887854,2.958933,2.501452,4.114600,1.997951,2.076782,3.142584,3.540576,5.622199
SLC39A6,0.223141,0.414837,0.207165,0.758208,0.552769,0.483878,0.343637,0.272967,0.997367,1.978218
CTSC,0.901751,2.988974,1.829729,1.145382,4.050639,1.284788,1.722533,0.597943,1.556963,3.001549


In [57]:
(marker_genes.isin(df_Puram_mean.index)==True).sum()

880

In [64]:
df_Puram_mean.index.to_list()

['C9orf152',
 'RPS11',
 'ELMO2',
 'CREB3L1',
 'PNMA1',
 'MMP2',
 'TMEM216',
 'TRAF3IP2-AS1',
 'LRRC37A5P',
 'LOC653712',
 'C10orf90',
 'ZHX3',
 'ERCC5',
 'GPR98',
 'RXFP3',
 'CTAGE10P',
 'APBB2',
 'KLHL13',
 'PDCL3',
 'AEN',
 'FRG2',
 'DECR1',
 'SALL1',
 'GGT3P',
 'CADM4',
 'RPS18',
 'SLC10A7',
 'CFHR5',
 'OR2K2',
 'BRIX1',
 'LMAN1',
 'CHD8',
 'SUMO1',
 'MIR941-2',
 'GP1BA',
 'BSN-AS2',
 'BOLA3-AS1',
 'UQCR11',
 'DDB1',
 'MYO9B',
 'MMP7',
 'CRNKL1',
 'XAB2',
 'RTN1',
 'KLHL14',
 'ADAM21P1',
 'TBX10',
 'ZBTB12',
 'UTY',
 'CENPQ',
 'ZAR1L',
 'DTNBP1',
 'KBTBD8',
 'ZEB1',
 'ZG16',
 'PRKAG2-AS1',
 'FAT4',
 'LOC100216546',
 'MIER1',
 'CHD9',
 'STK16',
 'ARID3C',
 'TOB2',
 'INIP',
 'BANK1',
 'OR2V2',
 'SFTA2',
 'GRM2',
 'PROSC',
 'SPIN2B',
 'PIR',
 'IPO9',
 'FLJ26850',
 'EVC',
 'CXCL13',
 'KIAA1199',
 'EHHADH-AS1',
 'SORL1',
 'NAT10',
 'CHD1',
 'SYN3',
 'SLC22A2',
 'SERPINF1',
 'WDR34',
 'OR7A17',
 'POTED',
 'EPT1',
 'LHB',
 'TAOK3',
 'STK25',
 'CDC25B',
 'BMP3',
 'TMEM180',
 'MAP1LC3C',
 'M

In [59]:
'SFTA2' in df_shared_mean.index

False

In [61]:
'SFTA2' in marker_genes

False

In [4]:
df_TCGA_shared

,0,6,7,8,10,12,13,16,17,18,20,22,23,24,25,26,28
AREG,218.775090,78.13442,0.00000,30.09585,66.871320,84.348800,168.346700,449.64160,686.85860,688.41724,90.370380,212.65027,14.244100,167.200200,830.91920,270.71954,409.97540
QPCTL,289.076350,237.98935,2.66097,101.38164,73.388010,136.335500,54.209880,533.28320,268.94766,401.33150,333.781700,96.20364,34.601234,84.978516,497.60614,260.82324,346.22168
CNN3,147.127440,914.64370,27.03170,273.27316,155.587620,43.292324,108.880470,1093.32860,1417.55380,927.66266,828.252000,510.68524,0.000000,575.223700,2048.17380,1599.81350,2626.36450
MTRNR2L6,671.693000,939.34094,7.06690,251.27254,242.529560,316.656040,201.426300,1414.46350,1477.01710,1467.85110,1762.216700,825.36540,299.477400,548.414730,3840.45000,2390.72780,3591.45500
PTPN1,283.433700,207.85945,3.61530,44.30932,128.515720,107.859580,157.980830,368.82138,500.62790,429.32742,364.492520,193.16628,44.852200,159.850360,749.53280,316.16910,669.48114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOXP1,522.212770,519.47284,8.76590,270.51523,102.644196,273.455380,212.604430,973.37665,539.08680,853.98930,314.579380,186.06172,46.034360,278.852050,1270.30960,536.54280,1039.81290
RABAC1,484.293640,838.13170,4.33760,178.75845,328.779970,180.154500,157.988020,1451.76440,1335.02880,1437.67490,1531.085100,515.56750,121.890300,478.761350,2308.32840,1064.41670,2170.86940
PTGS1,28.874298,50.79470,0.00000,28.10949,28.099714,0.137403,31.529453,148.61210,375.19850,136.91693,8.388001,158.25346,0.000000,203.309310,200.55597,100.36076,283.33344
ZNF793,248.700580,211.03482,2.46182,104.48741,63.530834,156.765950,52.710938,475.52510,215.03966,351.34436,303.478850,79.37739,35.850800,67.430690,485.51733,256.58210,324.74176


In [63]:
# writing a list of all DEGs
import csv
with open("compare_genes_test.txt",'w') as f:
    write = csv.writer(f)
    write.writerow(df_Puram_mean.index.tolist())

In [5]:
df_shared_mean

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,T cell,myocyte,other,tumor
AREG,1.248354,1.327800,0.087278,0.028747,2.251044,2.637694,0.764705,0.384740,0.989876,1.242421
QPCTL,0.802654,0.761660,0.925610,0.765424,0.636664,0.910067,1.116926,0.682296,0.967251,0.577489
CNN3,0.014843,0.962302,2.804872,2.759271,0.232031,0.092085,0.053545,1.463287,1.925831,1.733607
MTRNR2L6,2.376320,2.146333,2.274464,2.592525,2.086993,2.113999,1.957030,2.438877,2.289440,2.409220
PTPN1,1.810149,2.196046,0.710466,0.832199,1.729778,0.826140,0.877162,0.317696,0.670670,1.034497
...,...,...,...,...,...,...,...,...,...,...
FOXP1,0.509221,2.284009,1.939652,1.537203,1.119101,2.279452,1.532416,0.239747,1.461023,0.998507
RABAC1,2.723898,1.194062,2.216930,2.199098,1.612882,1.344311,1.321400,1.529787,1.928181,2.106459
PTGS1,0.074091,0.000000,0.153627,0.464268,1.180230,2.493842,0.009088,0.016448,0.141811,0.506399
ZNF793,0.689719,0.780146,0.864382,0.720488,0.611700,0.775994,1.084352,0.884953,0.875415,0.511459


In [6]:
df_shared_std

,B cell,Dendritic,Endothelial,Fibroblast,Macrophage,Mast,T cell,myocyte,other,tumor
AREG,2.527448,2.783769,0.473254,0.301962,3.768015,3.394832,2.065681,1.172476,2.057478,2.015638
QPCTL,0.649059,0.374258,0.743219,0.613191,0.329033,0.720361,0.644929,0.799138,0.926944,0.520956
CNN3,0.082777,2.058329,2.096504,2.223280,0.801207,0.458830,0.404193,2.471317,2.809141,2.032585
MTRNR2L6,2.350267,1.704151,1.594141,1.506708,1.672132,1.805756,1.488867,2.419163,2.255698,1.664874
PTPN1,2.250812,1.999035,1.366978,1.559267,1.863888,1.474077,1.896134,1.011864,1.336831,1.301459
...,...,...,...,...,...,...,...,...,...,...
FOXP1,1.229674,1.781295,1.905105,1.902467,1.303177,1.833933,2.299160,0.576199,2.024939,1.173314
RABAC1,2.614484,2.168226,2.874448,3.007220,2.257725,2.534091,2.732818,2.093489,2.915351,2.120058
PTGS1,0.455972,0.000000,0.650487,1.246237,1.923792,2.439153,0.195719,0.049978,0.661601,0.994970
ZNF793,0.341631,0.402770,0.516331,0.398696,0.525262,0.387454,0.469683,0.759592,0.777809,0.396856


In [7]:
df_TCGA_shared.to_numpy().shape

(880, 17)

In [8]:
hyperpars = {
    'Alpha': [1, 10],
    'Alpha0': [0.1, 1, 5],
    'Kappa0': [1, 0.5, 0.1],
    'SY': [1,0.3,0.5],
}

Nrep=3
Nrepfinal=10
Njob=10

In [10]:
print("start BLADE!")
Y = df_TCGA_shared.to_numpy()
mean = df_shared_mean.to_numpy() 
sd = df_shared_std.to_numpy() 

outfile = '/home/cke/BLADE/data/Puramfiltered_pseudobulk_BLADEout_2.pickle'

final_obj, best_obj, best_set, outs = Framework(
    mean, sd, Y,
    Alphas=hyperpars['Alpha'], Alpha0s=hyperpars['Alpha0'], 
    Kappa0s=hyperpars['Kappa0'], SYs=hyperpars['SY'],
    Nrep=Nrep, Njob=Njob, Nrepfinal=Nrepfinal)

pickle.dump(
    {
        'final_obj': final_obj,
        'best_obj': best_obj,
        'best_set': best_set,
        'outs' : outs
    }, open(outfile, 'wb')
    )

# In[ ]:

start BLADE!
all of 880 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:    1.2s
[Parallel(n_jobs=10)]: Done   2 out of  17 | elapsed:    1.2s remaining:    9.1s
[Parallel(n_jobs=10)]: Done   4 out of  17 | elapsed:    1.2s remaining:    4.0s
[Parallel(n_jobs=10)]: Done   6 out of  17 | elapsed:    1.2s remaining:    2.3s
[Parallel(n_jobs=10)]: Done   8 out of  17 | elapsed:    1.3s remaining:    1.4s
[Parallel(n_jobs=10)]: Done  10 out of  17 | elapsed:    1.4s remaining:    1.0s
[Parallel(n_jobs=10)]: Done  12 out of  17 | elapsed:    1.4s remaining:    0.6s
[Parallel(n_jobs=10)]: Done  14 out of  17 | elapsed:    1.4s remaining:    0.3s
[Parallel(n_jobs=10)]: Done  17 out of  17 | elapsed:    1.7s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.


No feature filtering is done (fsel = 0)


/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarnin

/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarnin

Done optimization, elapsed time (min): 16.853661664326985


FileNotFoundError: [Errno 2] No such file or directory: './BLADE/data/Puramfiltered_pseudobulk_BLADEout_2.pickle'

In [46]:
outfile = '/home/cke/BLADE/data/Puramfiltered_pseudobulk_BLADEout_2.pickle'

pickle.dump(
    {
        'final_obj': final_obj,
        'best_obj': best_obj,
        'best_set': best_set,
        'outs' : outs
    }, open(outfile, 'wb')
    )

Given the configuration above, BLADE is applied to each of the simulation dataset created previously.  

BLADE produce several outcomes:
- `final_obj`: final BLADE object with optimized variational parameters
- `best_obj`: BLADE object trained with the best parameter set found by the Empirical Bayes framework. Empirical Bayes framework is applied after selecting a subset of samples (5 samples; indicated by `Ind_sample` below), and thus the outcome contains only 5 samples. If `Nsample` <= 5, `final_obj` is identical to `best_obj`.
- `best_set`: Best parameter set defined by Empirical Bayes framework.
- `outs`: Outcome of BLADE for every possible combination of hyperparameters, used in the Empirical Bayes framework. 


- There are nan in mean and std matrix! NAs are filled with 0?

full tumor type setup:
- ngenes = 21706 common genes
- ncells = 24, including all 16 tumor types
- nsample = 546


simple tumor type setup:
- ngenes = 21706 common genes
- ncells = 10, all tumor types are merged, including one NA type?
- nsample = 546
- marker genes = 900 (including 9 genes not shared)


### test run with a small subset of scrna reference, take randomly 1000 genes from both ref and bulk counts
> reduced hyperparam combinations

In [ ]:
hyperpars_test = {
    'Alpha': [1, 10],
    'Alpha0': [0.1, 1, 5],
    'Kappa0': [1, 0.5, 0.1],
    'SY': [1,0.3,0.5],
}

Nrep=3
Nrepfinal=10
Njob=10

df_mean_sample = df_shared_mean.sample(n=2000)
df_std_sample = df_shared_std[df_shared_std.index.isin(df_mean_sample.index)]
Y_sample = df_TCGA_shared[df_TCGA_shared.index.isin(df_mean_sample.index)]

Y = Y_sample.to_numpy()
mean = df_mean_sample.to_numpy() 
sd = df_std_sample.to_numpy()

outfile = './BLADE/data/PuramTCGA_BLADE_Sample2000.pickle'

final_obj, best_obj, best_set, outs = Framework(
    mean, sd, Y,
    Alphas=hyperpars_test['Alpha'], Alpha0s=hyperpars_test['Alpha0'], 
    Kappa0s=hyperpars_test['Kappa0'], SYs=hyperpars_test['SY'],
    Nrep=Nrep, Njob=Njob, Nrepfinal=Nrepfinal)

pickle.dump(
    {
        'final_obj': final_obj,
        'best_obj': best_obj,
        'best_set': best_set,
        'outs' : outs
    }, open(outfile, 'wb')
    )


all of 2000 genes are used for optimization.
All samples are used during the optimization.
Initialization with Support vector regression


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   5 tasks      | elapsed:    7.1s
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    8.0s
[Parallel(n_jobs=10)]: Done  21 tasks      | elapsed:    8.7s
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.9s
[Parallel(n_jobs=10)]: Done  41 tasks      | elapsed:   10.4s
[Parallel(n_jobs=10)]: Done  52 tasks      | elapsed:   11.2s
[Parallel(n_jobs=10)]: Done  65 tasks      | elapsed:   12.0s
[Parallel(n_jobs=10)]: Done  78 tasks      | elapsed:   12.7s
[Parallel(n_jobs=10)]: Done  93 tasks      | elapsed:   14.1s
[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   15.0s
[Parallel(n_jobs=10)]: Done 125 tasks      | elapsed:   16.5s
[Parallel(n_jobs=10)]: Done 142 tasks      | elapsed:   17.8s
[Parallel(n_jobs=10)]: Done 161 tasks      | elapsed:   19.1s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   20.4s
[Parallel(n_jobs=10)]: Done 201 tasks      | elapsed:  

No feature filtering is done (fsel = 0)


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs

/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarning: invalid value encountered in double_scalars
  return PX+PY+PF-QX-QF
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:809: RuntimeWarning: divide by zero encountered in log
  return -self.Nsample*np.sum(np.log(Omega))
/home/cke/miniconda3/envs/BLADE/lib/python3.10/site-packages/Deconvolution/BLADE.py:863: RuntimeWarnin

## Results

In [65]:
BLADE_out = pickle.load(open("/home/cke/BLADE/data/Puramfiltered_pseudobulk_BLADEout_2.pickle", 'rb'))

In [66]:
obj = BLADE_out['final_obj']
    
outcomes = {
    'BLADE': {
        'Fraction': t(obj.ExpF(obj.Beta)), 
        'Signature': np.mean(obj.Nu, 0), #group mode purification
        'HighRes': obj.Nu                #highresolution mode purification
    }}

In [67]:
obj

In [68]:
outcomes['BLADE']['HighRes']

array([[[ 1.26149499e+00,  1.32880508e+00,  8.74101227e-02, ...,
          3.84936899e-01,  9.92568033e-01,  1.24586847e+00],
        [ 8.03453672e-01,  7.61808350e-01,  9.25836741e-01, ...,
          6.82371711e-01,  9.68220959e-01,  5.77717040e-01],
        [ 1.54093262e-02,  9.63745928e-01,  2.80614574e+00, ...,
          1.45993435e+00,  1.91034487e+00,  1.73744816e+00],
        ...,
        [ 7.55731232e-02, -1.09473318e-02,  1.53909305e-01, ...,
          1.65810050e-02,  1.43240653e-01,  5.07403116e-01],
        [ 6.90456164e-01,  7.80346420e-01,  8.64612947e-01, ...,
          8.85071832e-01,  8.76323928e-01,  5.11718603e-01],
        [ 2.63344482e+00,  3.07816053e+00,  2.98704538e+00, ...,
          2.39721743e+00,  2.55145991e+00,  2.76641960e+00]],

       [[ 1.26111156e+00,  1.32652245e+00,  8.77368537e-02, ...,
          3.85005429e-01,  9.91381733e-01,  1.24930893e+00],
        [ 8.03734953e-01,  7.61840689e-01,  9.26076117e-01, ...,
          6.82393428e-01,  9.67969635e

In [69]:
filtered_celltypefrac_BLADE = pd.DataFrame(outcomes['BLADE']['Fraction'])
filtered_celltypefrac_BLADE

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.203741,0.231715,0.216827,0.238812,0.336297,0.205316,0.228220,0.236456,0.200568,0.234780,0.185197,0.195042,0.206827,0.219747,0.234669,0.199102,0.204632
1,0.053973,0.048428,0.033642,0.057604,0.082735,0.050630,0.095307,0.048152,0.031864,0.051436,0.041525,0.033669,0.052700,0.057209,0.048889,0.049616,0.053465
2,0.058477,0.073942,0.060768,0.071693,0.068023,0.055627,0.072331,0.066484,0.060351,0.072212,0.057872,0.055522,0.057286,0.079907,0.071840,0.106321,0.077223
3,0.091541,0.148503,0.122186,0.128048,0.105399,0.108828,0.101535,0.127919,0.099956,0.112251,0.154354,0.114902,0.113316,0.210472,0.174017,0.145528,0.209253
4,0.047581,0.044249,0.033974,0.047277,0.053619,0.043822,0.056773,0.039281,0.035529,0.045587,0.035714,0.034069,0.034570,0.045681,0.047626,0.045624,0.043039
5,0.034685,0.019134,0.017295,0.031519,0.038566,0.031544,0.040986,0.026890,0.021181,0.029117,0.015213,0.019423,0.032886,0.023683,0.022886,0.027511,0.023305
6,0.264119,0.102587,0.131659,0.164655,0.098735,0.270693,0.127745,0.215020,0.104498,0.161110,0.073403,0.170673,0.257521,0.110517,0.118776,0.109683,0.117511
7,0.017241,0.025758,0.019174,0.018043,0.026430,0.018967,0.029295,0.027741,0.022490,0.025750,0.023600,0.021836,0.022527,0.026305,0.031234,0.029980,0.035647
8,0.127103,0.104931,0.116625,0.109716,0.088016,0.094922,0.137421,0.091571,0.089681,0.130692,0.078010,0.097335,0.093416,0.101992,0.077124,0.102496,0.091508
9,0.101539,0.200753,0.247850,0.132632,0.102179,0.119650,0.110388,0.120487,0.333881,0.137065,0.335112,0.257529,0.128950,0.124487,0.172939,0.184139,0.144418


In [70]:
filtered_celltypefrac_BLADE.columns = df_TCGA_shared.columns

In [71]:
filtered_celltypefrac_BLADE.index = df_shared_mean.columns

In [72]:
list_ind = filtered_celltypefrac_BLADE.index.tolist()
# list_ind[6] = 'other'
filtered_celltypefrac_BLADE.index=list_ind

In [73]:
filtered_celltypefrac_BLADE

,0,6,7,8,10,12,13,16,17,18,20,22,23,24,25,26,28
B cell,0.203741,0.231715,0.216827,0.238812,0.336297,0.205316,0.228220,0.236456,0.200568,0.234780,0.185197,0.195042,0.206827,0.219747,0.234669,0.199102,0.204632
Dendritic,0.053973,0.048428,0.033642,0.057604,0.082735,0.050630,0.095307,0.048152,0.031864,0.051436,0.041525,0.033669,0.052700,0.057209,0.048889,0.049616,0.053465
Endothelial,0.058477,0.073942,0.060768,0.071693,0.068023,0.055627,0.072331,0.066484,0.060351,0.072212,0.057872,0.055522,0.057286,0.079907,0.071840,0.106321,0.077223
Fibroblast,0.091541,0.148503,0.122186,0.128048,0.105399,0.108828,0.101535,0.127919,0.099956,0.112251,0.154354,0.114902,0.113316,0.210472,0.174017,0.145528,0.209253
Macrophage,0.047581,0.044249,0.033974,0.047277,0.053619,0.043822,0.056773,0.039281,0.035529,0.045587,0.035714,0.034069,0.034570,0.045681,0.047626,0.045624,0.043039
Mast,0.034685,0.019134,0.017295,0.031519,0.038566,0.031544,0.040986,0.026890,0.021181,0.029117,0.015213,0.019423,0.032886,0.023683,0.022886,0.027511,0.023305
T cell,0.264119,0.102587,0.131659,0.164655,0.098735,0.270693,0.127745,0.215020,0.104498,0.161110,0.073403,0.170673,0.257521,0.110517,0.118776,0.109683,0.117511
myocyte,0.017241,0.025758,0.019174,0.018043,0.026430,0.018967,0.029295,0.027741,0.022490,0.025750,0.023600,0.021836,0.022527,0.026305,0.031234,0.029980,0.035647
other,0.127103,0.104931,0.116625,0.109716,0.088016,0.094922,0.137421,0.091571,0.089681,0.130692,0.078010,0.097335,0.093416,0.101992,0.077124,0.102496,0.091508
tumor,0.101539,0.200753,0.247850,0.132632,0.102179,0.119650,0.110388,0.120487,0.333881,0.137065,0.335112,0.257529,0.128950,0.124487,0.172939,0.184139,0.144418


In [74]:
# ignore this block and above
# use code in runMuSiC to store and access celltypefrac file
filtered_celltypefrac_BLADE.T.to_csv("/home/cke/BLADE/data/filtered_celltypefrac_BLADE_pseudobulk.csv")